In [24]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mlos_core.optimizers
import ConfigSpace as CS
from ConfigSpace import UniformIntegerHyperparameter
from ConfigSpace import UniformFloatHyperparameter
from ConfigSpace import CategoricalHyperparameter

In [33]:
def cost_model(index):
    performance_data = pd.read_csv('../../build/performance/performance_metrics.csv', header=None, names=["empty_reads", "non_empty_reads", "range_reads", "writes"])
    return performance_data.iloc[index:].mean().sum()

In [34]:
default_values = {
    "max_background_jobs": 2,
    "max_subcompactions": 1,
    "max_write_buffer_number":2,
    "level0_slowdown_writes_trigger": 20,
    "level0_stop_writes_trigger": 36,
    "target_file_size_base": 64 * 1048576,
    "target_file_size_multiplier": 1,
    "max_bytes_for_level_multiplier": 10,
    "write_buffer_size": 64 << 20,
    "level0_file_num_compaction_trigger": 4,
    "max_bytes_for_level_base": 256 * 1048576,
    "score":0
    
}


In [35]:
input_space = CS.ConfigurationSpace(seed=1234)

max_open_files=CS.Integer("max_open_files",(-1,1000),default=-1)
max_total_wal_size=CS.Integer("max_total_wal_size",(0,1000))
delete_obsolete_files_period_micros=CS.Integer("delete_obsolete_files_period_micros",(1,1000 * 60 * 60 * 1000000),default=6 * 60 * 60 * 1000000)

max_background_jobs=CS.Integer("max_background_jobs", (1,20), default=default_values["max_background_jobs"])
max_subcompactions=CS.Integer("max_subcompactions", (1,20), default=default_values["max_subcompactions"])
compaction_readahead_size=CS.Integer("compaction_readahead_size",(0,1000 * 1024 * 1024),default=2 * 1024 * 1024)

writable_file_max_buffer_size=CS.Integer("writable_file_max_buffer_size",(0,1000*1024*1024),default=1024 * 1024)
delayed_write_rate=CS.Integer("delayed_write_rate",(0,1000*1024*1024),default=0)
avoid_flush_during_shutdown=CS.Categorical("avoid_flush_during_shutdown", ["true","false"], default="true") 

max_write_buffer_number=CS.Integer("max_write_buffer_number", (1,20), default=default_values["max_write_buffer_number"])
inplace_update_num_locks=CS.Integer("inplace_update_num_locks",(0,100000),default=10000)
memtable_prefix_bloom_size_ratio=CS.Float("memtable_prefix_bloom_size_ratio",(0.0,1.0),default=0)

memtable_whole_key_filtering=CS.Categorical("memtable_whole_key_filtering",["true","false"], default="false") 
memtable_huge_page_size=CS.Integer("memtable_huge_page_size",(0,1000 * 1024 * 1024),default=0)
arena_block_size=CS.Integer("arena_block_size",(0, 2<<30),default=0)

# compression_per_level=CS.("compression_per_level")
level0_slowdown_writes_trigger=CS.Integer("level0_slowdown_writes_trigger", (1,25), default=default_values["level0_slowdown_writes_trigger"]) 
level0_stop_writes_trigger=CS.Integer("level0_stop_writes_trigger", (1,50), default=default_values["level0_stop_writes_trigger"]) 

target_file_size_base=CS.Integer("target_file_size_base", (1,500 * 1048576), default=default_values["target_file_size_base"])
target_file_size_multiplier=CS.Integer("target_file_size_multiplier", (1,20), default=default_values["target_file_size_multiplier"])
max_bytes_for_level_multiplier=CS.Float("max_bytes_for_level_multiplier", (2,20.0), default=default_values["max_bytes_for_level_multiplier"])

write_buffer_size=CS.Float("write_buffer_size",(20<<20,80<<20),default=default_values["write_buffer_size"])
level0_file_num_compaction_trigger=CS.Integer("level0_file_num_compaction_trigger",(-1,10),default=default_values["level0_file_num_compaction_trigger"])
max_bytes_for_level_base=CS.Integer("max_bytes_for_level_base",(128 * 1048576,256 * 1048576),default=default_values["max_bytes_for_level_base"])

# max_bytes_for_level_multiplier_additional=CS.("max_bytes_for_level_multiplier_additional")
# max_compaction_bytes=CS.Integer("max_compaction_bytes")
# soft_pending_compaction_bytes_limit=CS.Integer("soft_pending_compaction_bytes_limit")

# hard_pending_compaction_bytes_limit=CS.Integer("hard_pending_compaction_bytes_limit")
# compaction_options_universal=CS.("compaction_options_universal")
# compaction_options_fifo=CS.("compaction_options_fifo")

max_sequential_skip_in_iterations=CS.Integer("max_sequential_skip_in_iterations",(0,999999),default=8)
max_successive_merges=CS.Integer("max_successive_merges",(0,1000),default=0)
check_flush_compaction_key_order=CS.Categorical("check_flush_compaction_key_order",["true","false"], default="true") 

paranoid_file_checks=CS.Categorical("paranoid_file_checks",["true","false"], default="false") 
report_bg_io_stats=CS.Categorical("report_bg_io_stats",["true","false"], default="false") 
# ttl=CS.Integer("ttl")

# periodic_compaction_seconds=CS.Integer("periodic_compaction_seconds")
# bottommost_temperature=CS.("bottommost_temperature")
enable_blob_files=CS.Categorical("enable_blob_files",["true","false"], default="false") 

min_blob_size=CS.Integer("min_blob_size",(0,1024),default=0)
# blob_file_size=CS.Integer("blob_file_size")
# blob_compression_type=CS.("blob_compression_type")

enable_blob_garbage_collection=CS.Categorical("enable_blob_garbage_collection",["true","false"], default="false") 
blob_garbage_collection_age_cutoff=CS.Float("blob_garbage_collection_age_cutoff",(0.0,1.0),default=0.25)
blob_garbage_collection_force_threshold=CS.Float("blob_garbage_collection_force_threshold",(0.0,1.0),default=1.00)

blob_compaction_readahead_size=CS.Integer("blob_compaction_readahead_size",(0,999999),default=0)
blob_file_starting_level=CS.Integer("blob_file_starting_level",(0,20),default=0)

In [36]:
input_space.add_hyperparameters([max_background_jobs,
                                  max_subcompactions,
                                 max_write_buffer_number,
                                  level0_slowdown_writes_trigger,
                                level0_stop_writes_trigger,
                                 target_file_size_base,
                                target_file_size_multiplier,
                                max_bytes_for_level_multiplier,
                                 write_buffer_size,
                                 level0_file_num_compaction_trigger,
                                 max_bytes_for_level_base])
                                 
#                                 max_open_files,
#                                  max_total_wal_size,
##                                  delete_obsolete_files_period_micros,
                               
#                                 compaction_readahead_size,
#                                 writable_file_max_buffer_size,
#                                 delayed_write_rate,
#                                 avoid_flush_during_shutdown,
                                
#                                 inplace_update_num_locks,
#                                 memtable_prefix_bloom_size_ratio,
#                                 memtable_whole_key_filtering,
#                                 memtable_huge_page_size,
#                                 arena_block_size,
                               
                                
#                                 max_sequential_skip_in_iterations,
#                                 max_successive_merges,
#                                 check_flush_compaction_key_order,
#                                 paranoid_file_checks,
#                                 report_bg_io_stats,
#                                 enable_blob_files,
#                                 min_blob_size,
#                                 enable_blob_garbage_collection,
#                                 blob_garbage_collection_age_cutoff,
#                                 blob_garbage_collection_force_threshold,
#                                 blob_compaction_readahead_size,
#                                 blob_file_starting_level])

[max_background_jobs, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_subcompactions, Type: UniformInteger, Range: [1, 20], Default: 1,
 max_write_buffer_number, Type: UniformInteger, Range: [1, 20], Default: 2,
 level0_slowdown_writes_trigger, Type: UniformInteger, Range: [1, 25], Default: 20,
 level0_stop_writes_trigger, Type: UniformInteger, Range: [1, 50], Default: 36,
 target_file_size_base, Type: UniformInteger, Range: [1, 524288000], Default: 67108864,
 target_file_size_multiplier, Type: UniformInteger, Range: [1, 20], Default: 1,
 max_bytes_for_level_multiplier, Type: UniformFloat, Range: [2.0, 20.0], Default: 10.0,
 write_buffer_size, Type: UniformFloat, Range: [20971520.0, 83886080.0], Default: 67108864.0,
 level0_file_num_compaction_trigger, Type: UniformInteger, Range: [-1, 10], Default: 4,
 max_bytes_for_level_base, Type: UniformInteger, Range: [134217728, 268435456], Default: 268435456]

In [37]:
#optimizer = mlos_core.optimizers.RandomOptimizer(parameter_space=input_space)

#optimizer = mlos_core.optimizers.FlamlOptimizer(parameter_space=input_space)

optimizer = mlos_core.optimizers.SmacOptimizer(parameter_space=input_space) # , seed=42, n_random_init=20)

In [115]:
optimizer

SmacOptimizer(space_adapter=None)

In [39]:
import os
import time
import sys
def pass_values_to_interface(new_val):
    pipe_path = '../../build/passing_params_pipe'
    if not os.path.exists(pipe_path):
        os.mkfifo(pipe_path)
    pipe_fd = os.open(pipe_path, os.O_RDWR | os.O_TRUNC)
    data =''
    try:
        for x in new_val:
            data+='\n' + x + '=' + str(new_val[x].iloc[0])
        data=data.encode('utf-8')
        os.write(pipe_fd, data)
        sys.stdout.flush()
    except KeyboardInterrupt:
        pass
#     finally:
#         os.close(pipe_fd)


In [40]:
def read_epochs_pipe():
    while True:
        epochs_pipe_path = '../../build/passing_epochs'
        if os.path.exists(epochs_pipe_path):
            epochs_pipe_fd = os.open(epochs_pipe_path, os.O_RDONLY | os.O_TRUNC)
            try:
                buffer_size = 10000
                buffer = os.read(epochs_pipe_fd, buffer_size).decode('utf-8')
                if buffer:
                    reported_epochs = int(buffer)
                    print(f"Reported epochs: {reported_epochs}")
                    return reported_epochs
            except OSError as e:
                if e.errno == os.errno.EAGAIN or e.errno == os.errno.EWOULDBLOCK:
                    pass
                else:
                    raise
            finally:
                os.close(epochs_pipe_fd)
        else:
            pass
    


In [118]:
restart_indexes=[]
def run_optimization():
    suggested_value = optimizer.suggest()
    pass_values_to_interface(suggested_value)
    index=read_epochs_pipe()
    time.sleep(5)
    if index<0:
        restart_indexes.append(-index)
        target_value=float('inf')
    else:
        target_value = cost_model(index)
    optimizer.register(suggested_value, pd.Series([target_value]))


n_iterations = 50
for i in range(n_iterations):
    print("epoch:",i)
    run_optimization()

epoch: 0
Reported epochs: -15
epoch: 1
Reported epochs: -19
epoch: 2
Reported epochs: -41
epoch: 3
Reported epochs: -57
epoch: 4
Reported epochs: -69
epoch: 5


KeyboardInterrupt: 

In [25]:
observations_df = optimizer.get_observations()

In [96]:
observations_df

In [97]:
delta_df = observations_df.copy()
for column in delta_df.columns:
    default_value = default_values[column]
    delta_df[column] = delta_df[column] - default_value

In [98]:
delta_df

,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,level0_stop_writes_trigger,max_background_jobs,max_bytes_for_level_base,max_bytes_for_level_multiplier,max_subcompactions,max_write_buffer_number,target_file_size_base,target_file_size_multiplier,write_buffer_size,score
0,2,1,11,17,-124698764,-0.022787,5,13,43722959,3,-4.191399e+07,15.381579
0,-2,-18,-12,2,-17183175,5.942768,18,6,373973483,15,-5.721993e+06,15.597403
0,-3,-4,2,9,-64324988,3.436062,4,2,311371948,10,1.447204e+07,16.362319
0,6,-11,-27,4,-96302129,-6.480219,11,15,116288409,7,-2.948443e+07,inf
0,4,-1,-20,6,-78529936,-3.361093,0,0,219367849,19,6.249891e+06,10.552083
0,-5,-8,7,11,-46576860,9.350033,12,18,151284373,1,-2.243107e+07,42.916667
0,-1,4,-30,0,-1376359,2.276874,9,9,-56465192,7,-3.369996e+07,9.041237
0,3,-14,-8,15,-108917036,-5.393835,16,7,400737269,14,-1.276811e+07,9.940594
0,3,-7,-35,13,-26868414,-7.596247,2,-1,243717966,9,-7.136720e+06,inf
0,0,-13,-7,8,-133237747,4.552482,14,17,175642505,11,-3.534152e+07,9.323810


In [99]:
optimizer.get_best_observation()

In [100]:
pass_values_to_interface(optimizer.get_best_observation())

In [102]:
import pandas as pd
import plotly.express as px

data_url = '../../build/performance/performance_metrics.csv'
df = pd.read_csv(data_url, header=None, names=["Empty Reads", "Non-Empty Reads", "Range Reads", "Writes"]).iloc[-1000:]

avg_latency =df.iloc[-1000:].sum().mean()
print("avg latency,"avg_latency)
# Plot Empty Reads\n",
fig_empty_reads = px.line(df, x=df.index, y="Empty Reads", title="Empty Reads")
fig_non_empty_reads = px.line(df, x=df.index, y="Non-Empty Reads", title="Non-Empty Reads")
fig_range_reads = px.line(df, x=df.index, y="Range Reads", title="Range Reads")
fig_writes = px.line(df, x=df.index, y="Writes", title="Writes")
                     
# for restart_index in restart_indexes:
#     fig_empty_reads.add_trace(go.Scatter(x=[restart_index], y=[df["Empty Reads"].iloc[restart_index]],
#                                          mode='markers', marker=dict(color='red'), name='Restart'))
#     fig_non_empty_reads.add_trace(go.Scatter(x=[restart_index], y=[df["Non-Empty Reads"].iloc[restart_index]],
#                                             mode='markers', marker=dict(color='red'), name='Restart'))
#     fig_range_reads.add_trace(go.Scatter(x=[restart_index], y=[df["Range Reads"].iloc[restart_index]],
#                                          mode='markers', marker=dict(color='red'), name='Restart'))
#     fig_writes.add_trace(go.Scatter(x=[restart_index], y=[df["Writes"].iloc[restart_index]],
#                                     mode='markers', marker=dict(color='red'), name='Restart'))


fig_empty_reads.update_xaxes(title_text='Iterations')
fig_empty_reads.update_yaxes(title_text='Time (ms)')
fig_non_empty_reads.update_xaxes(title_text='Iterations')
fig_non_empty_reads.update_yaxes(title_text='Time (ms)')
fig_range_reads.update_xaxes(title_text='Iterations')
fig_range_reads.update_yaxes(title_text='Time (ms)')
fig_writes.update_xaxes(title_text='Iterations')
fig_writes.update_yaxes(title_text='Time (ms)')

                     
fig_empty_reads.show()
fig_non_empty_reads.show()
fig_range_reads.show()
fig_writes.show()

fig_empty_reads.write_image("../../build/performance/empty_reads_graph.png")
fig_non_empty_reads.write_image("../../build/performance/non_empty_reads_graph.png")
fig_range_reads.write_image("../../build/performance/range_reads_graph.png")
fig_writes.write_image("../../build/performance/writes_graph.png")

python(29419) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
hyper_parameters_integer = {
    "max_open_files":[-1,10000, -1],
    "max_total_wal_size":[0, 100000, 0],
    "delete_obsolete_files_period_micros":[0, 6 * 60 * 60 * 1000000, 6 * 60 * 60 * 1000000],
    "max_background_jobs":[2, 10, 2],
    "max_subcompactions":[1, 100, 1],
    "compaction_readahead_size":[0, 1000000, 0],
    "writable_file_max_buffer_size":[0, 1024 * 1024, 1024 * 1024],
    "delayed_write_rate":[0, 1000000, 0],
    
    "max_write_buffer_number":[1, 10, 2],
    "inplace_update_num_locks":[0, 10000, 0],
    "memtable_huge_page_size":[0, 1000000, 0],
    "arena_block_size":[0, 1000000, 0],
    "level0_slowdown_writes_trigger":[0, 100, 20],
    "level0_stop_writes_trigger":[0, 100, 36],
    "target_file_size_base":[0, 100000000, 64 * 1048576],
    "target_file_size_multiplier":[0, 100, 1],
    "max_compaction_bytes":[0, 1000000000, 64 * 1048576 * 25],
    "soft_pending_compaction_bytes_limit":[0, 1000000000, 64 * 1073741824],
    "hard_pending_compaction_bytes_limit":[0, 1000000000, 256 * 1073741824]                          
}
                                           
hyper_parameters_float ={
    "memtable_prefix_bloom_size_ratio":[0.0, 1.0, 0.0],
    "max_bytes_for_level_multiplier":[0.0, 100.0, 10.0]
}
                                           
hyper_parameters_catagorical ={
    "avoid_flush_during_shutdown":[[True, False],False],  
    "memtable_whole_key_filtering":[[True, False],False],
   "compression_per_level":[["kNoCompression", "kSnappyCompression", "kZlibCompression", "kBZip2Compression", "kLZ4Compression", "kLZ4HCCompression", "kXpressCompression", "kZSTD"], "kNoCompression"]

}

In [1]:
import plotly.graph_objects as go
from tabulate import tabulate
import mlos_core.optimizers
import ConfigSpace as CS
from ConfigSpace import UniformIntegerHyperparameter
from ConfigSpace import UniformFloatHyperparameter
from ConfigSpace import CategoricalHyperparameter
import pandas as pd
import plotly.express as px
import os
import time
import sys
import logging

In [104]:
data_url = '../../build/performance/performance_metrics.csv'
df = pd.read_csv(data_url, header=None, names=["Empty Reads", "Non-Empty Reads", "Range Reads", "Writes"])
avg_latency =df.iloc[-1000:].sum().mean()
print("avg latency,"avg_latency)

In [105]:
df.shape

(2903, 4)

In [5]:
df.head()

,Empty Reads,Non-Empty Reads,Range Reads,Writes
0,0,16,1,0
1,0,6,1,0
2,0,6,1,0
3,0,7,1,0
4,0,7,1,3


In [107]:

# Plot Empty Reads\n",
fig_empty_reads = px.line(df, x=df.index, y="Empty Reads", title="Empty Reads")
fig_non_empty_reads = px.line(df, x=df.index, y="Non-Empty Reads", title="Non-Empty Reads")
fig_range_reads = px.line(df, x=df.index, y="Range Reads", title="Range Reads")
fig_writes = px.line(df, x=df.index, y="Writes", title="Writes")

# for restart_index in restart_indexes:
#     fig_empty_reads.add_trace(go.Scatter(x=[restart_index], y=[df["Empty Reads"].iloc[restart_index]],
#                                          mode='markers', marker=dict(color='red',size=10), legendgroup="group"))
#     fig_non_empty_reads.add_trace(go.Scatter(x=[restart_index], y=[df["Non-Empty Reads"].iloc[restart_index]],
#                                             mode='markers', marker=dict(color='red',size=10), showlegend=False))
#     fig_range_reads.add_trace(go.Scatter(x=[restart_index], y=[df["Range Reads"].iloc[restart_index]],
#                                          mode='markers', marker=dict(color='red',size=10), showlegend=False))
#     fig_writes.add_trace(go.Scatter(x=[restart_index], y=[df["Writes"].iloc[restart_index]],
#                                     mode='markers', marker=dict(color='red',size=10), showlegend=False))

In [108]:
fig_empty_reads.update_xaxes(title_text='Iterations')
fig_empty_reads.update_yaxes(title_text='Time (ms)')
fig_non_empty_reads.update_xaxes(title_text='Iterations')
fig_non_empty_reads.update_yaxes(title_text='Time (ms)')
fig_range_reads.update_xaxes(title_text='Iterations')
fig_range_reads.update_yaxes(title_text='Time (ms)')
fig_writes.update_xaxes(title_text='Iterations')
fig_writes.update_yaxes(title_text='Time (ms)')

                     
fig_empty_reads.show()
fig_non_empty_reads.show()
fig_range_reads.show()
fig_writes.show()

fig_empty_reads.write_image("../../build/performance/empty_reads_graph_no_tuning.png")
fig_non_empty_reads.write_image("../../build/performance/non_empty_reads_graph_no_tuning.png")
fig_range_reads.write_image("../../build/performance/range_reads_graph_no_tuning.png")
fig_writes.write_image("../../build/performance/writes_graph_no_tuning.png")
